# 7. Neural Machine Translation and Models with Attention

* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture9.pdf
* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture10.pdf
* https://arxiv.org/pdf/1409.0473.pdf

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
from copy import deepcopy
import os
flatten = lambda l: [item for sublist in l for item in sublist]

In [2]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [3]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [4]:
def pad_to_batch(batch,x_to_ix,y_to_ix):
    x,y = list(zip(*batch))
    max_x = max([s.size(1) for s in x])
    max_y = max([s.size(1) for s in y])
    x_p,y_p=[],[]
    for i in range(len(batch)):
        if x[i].size(1)<max_x:
            x_p.append(torch.cat([x[i],Variable(LongTensor([x_to_ix['<PAD>']]*(max_x-x[i].size(1)))).view(1,-1)],1))
        else:
            x_p.append(x[i])
        if y[i].size(1)<max_y:
            y_p.append(torch.cat([y[i],Variable(LongTensor([y_to_ix['<PAD>']]*(max_y-y[i].size(1)))).view(1,-1)],1))
        else:
            y_p.append(y[i])
    return torch.cat(x_p),torch.cat(y_p)

In [5]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if w in to_index.keys() else to_index["<unk>"], seq))
    return Variable(LongTensor(idxs))

### Data load and Preprocessing 

IWSLT'15 English-Vietnamese data(small)

In [6]:
source_corpus = open('./dataset/iwslt15/train.en','r',encoding='utf-8').readlines()
target_corpus = open('./dataset/iwslt15/train.vi','r',encoding='utf-8').readlines()

In [7]:
print(len(source_corpus)==len(target_corpus))
print(len(source_corpus))

True
133317


In [8]:
%%time
X_r,y_r=[],[] # raw

for parallel in zip(source_corpus,target_corpus):
    if parallel[1]=='\n':continue
    so,ta = parallel[0][:-1], parallel[1][:-1]
    if so.strip()=="" or ta.strip()=="": continue
    X_r.append(so.split())
    y_r.append(ta.split())

CPU times: user 864 ms, sys: 72 ms, total: 936 ms
Wall time: 936 ms


### Build Vocab

In [13]:
source_vocab = list(set(flatten(X_r)))
target_vocab = list(set(flatten(y_r)))
print(len(source_vocab),len(target_vocab))

54169 25615


In paper, they limit the source and target vocabulary to the most frequent 15,000 words. <br>
There are some problems of softmax operation's complexity to large target. You should determine this hyper-param in practice. 
We'll deal with this large vocab problem later.

In [15]:
VOCAB_LIMIT = 15000

In [16]:
source_vocab_count = Counter(flatten(X_r))
source_vocab, _ = list(zip(*source_vocab_count.most_common()[:VOCAB_LIMIT]))

target_vocab_count = Counter(flatten(y_r))
target_vocab, _  = list(zip(*target_vocab_count.most_common()[:VOCAB_LIMIT]))
print(len(source_vocab),len(target_vocab))

5000 5000


In [9]:
source_vocab = open('./dataset/iwslt15/vocab.en','r',encoding='utf-8').readlines()
source_vocab = [s[:-1] for s in source_vocab]
target_vocab = open('./dataset/iwslt15/vocab.vi','r',encoding='utf-8').readlines()
target_vocab = [t[:-1] for t in target_vocab]
print(len(source_vocab),len(target_vocab))

17191 7709


In [10]:
source2index = {'<PAD>':0,'<unk>':1,'<s>':2,'</s>':3}
for vo in source_vocab:
    source2index[vo]=len(source2index)
index2source = {v:k for k,v in source2index.items()}

target2index = {'<PAD>':0,'<unk>':1,'<s>':2,'</s>':3}
for vo in target_vocab:
    target2index[vo]=len(target2index)
index2target = {v:k for k,v in target2index.items()}

In [14]:
so_length_dist = Counter([len(x) for x in X_r])
ta_length_dist = Counter([len(y) for y in y_r])

In [15]:
print(so_length_dist.most_common()[:10],ta_length_dist.most_common()[:10])

[(10, 6305), (9, 6203), (11, 6188), (8, 6086), (12, 6035), (13, 5571), (14, 5426), (7, 5294), (15, 4996), (16, 4721)] [(12, 5180), (11, 5169), (13, 5053), (10, 5041), (14, 4951), (9, 4721), (15, 4720), (16, 4556), (17, 4398), (18, 4285)]


In [12]:
MAX_LENGTH=100 # because of unsufficient memory

In [13]:
%%time
X_p,y_p=[],[]

for so,ta in zip(X_r,y_r):
    X_p.append(prepare_sequence(so[:MAX_LENGTH],source2index).view(1,-1))
    y_p.append(prepare_sequence(ta[:MAX_LENGTH],target2index).view(1,-1))
    
train_data = list(zip(X_p,y_p))

CPU times: user 7.84 s, sys: 352 ms, total: 8.19 s
Wall time: 8.19 s


### Modeling 

In [16]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size,hidden_size, n_layers=1,bidirec=False):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        if bidirec:
            self.n_direction = 2 
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True,bidirectional=True)
        else:
            self.n_direction = 1
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True)
    
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers*self.n_direction,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
#         self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
#         self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
    
    def forward(self, inputs, input_masking=None):
        """
        inputs : B,T (LongTensor)
        input_masking : B,T (ByteTensor) if you don't use zero-padding, leave it at that
        """
        hidden = self.init_hidden(inputs)
        
        embedded = self.embedding(inputs)
        output, hidden = self.gru(embedded, hidden)

        real_context=[]
        
        if type(input_masking)==torch.autograd.variable.Variable:
            for i,o in enumerate(output): # B,T,D
                real_length = input_masking[i].data.tolist().count(0) # real length
                real_context.append(o[real_length-1])
            hidden = torch.cat(real_context).view(inputs.size(0),-1).unsqueeze(1)
        else:
            hidden = torch.cat(hidden,1).unsqueeze(1)
        
        return output, hidden

In [17]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, n_layers=1,dropout_p=0.3):
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        # Define the layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout_p)
        
        self.gru = nn.GRU(embedding_size+hidden_size, hidden_size, n_layers,batch_first=True)
        self.linear = nn.Linear(hidden_size*2, input_size)
        self.attn = nn.Linear(self.hidden_size,self.hidden_size) # Attention
        
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
#         self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
#         self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
        self.linear.weight = nn.init.xavier_uniform(self.linear.weight)
    
    
    def Attention(self, hidden, encoder_outputs, encoder_maskings):
        """
        hidden : 1,B,D
        encoder_outputs : B,T,D
        encoder_maskings : B,T # ByteTensor
        """
        
        hidden = hidden.squeeze(0).unsqueeze(2)  # 히든 : (1,B,D) -> (B,D,1)
        
        batch_size = encoder_outputs.size(0) # B
        max_len = encoder_outputs.size(1) # T
        energies = self.attn(encoder_outputs.contiguous().view(batch_size*max_len,-1)) # B*T,D -> B*T,D
        energies = energies.view(batch_size,max_len,-1) # B,T,D
        attn_energies = energies.bmm(hidden).transpose(1,2) # B,T,D * B,D,1 --> B,1,T
        
        if type(encoder_maskings)==torch.autograd.variable.Variable:
            attn_energies = attn_energies.squeeze(1).masked_fill(encoder_maskings,-1e12) # PAD masking

        alpha = F.softmax(attn_energies) # B,T
        alpha = alpha.unsqueeze(1) # B,1,T
        context = alpha.bmm(encoder_outputs) # B,1,T * B,T,D => B,1,D
        
        return context # B,1,D
    
    
    def forward(self,inputs,context,max_length,encoder_outputs,encoder_maskings=None,training=False):
        """
        inputs : B,1 (LongTensor, START SYMBOL)
        context : B,1,D (FloatTensor, Last encoder hidden state)
        encoder_outputs : B,T,D
        encoder_maskings : B,T # ByteTensor
        max_length : int, max length to decode
        training : bool, this is because adapt dropout only training step.
        """
        # Get the embedding of the current input word
        embedded = self.embedding(inputs)
        hidden = self.init_hidden(inputs)
        if training:
            embedded = self.dropout(embedded)
        
        decode=[]
        # Apply GRU to the output so far
        for i in range(max_length):

            _, hidden = self.gru(torch.cat((embedded,context),2), hidden) # h_t = f(h_{t-1},y_{t-1},c)
            concated = torch.cat((hidden,context.transpose(0,1)),2) # y_t = g(h_t,y_{t-1},c)
            score = self.linear(concated.squeeze(0))
            softmaxed = F.log_softmax(score)
            decode.append(softmaxed)
            decoded = softmaxed.max(1)[1]
            embedded = self.embedding(decoded).unsqueeze(1) # y_{t-1}
            if training:
                embedded = self.dropout(embedded)
            
            # compute next context vector using attention
            context = self.Attention(hidden[0], encoder_outputs,encoder_maskings) 
        #  column-wise concat, reshape!!
        scores = torch.cat(decode,1)
        return scores.view(inputs.size(0)*max_length,-1)

### Train 

It takes for a while...

In [19]:
STEP=50
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 256
LR = 1e-3

In [20]:
encoder = Encoder(len(source2index),EMBEDDING_SIZE,HIDDEN_SIZE,1,True)
decoder = Decoder(len(target2index),EMBEDDING_SIZE,HIDDEN_SIZE*2)
encoder.init_weight()
decoder.init_weight()

if USE_CUDA:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

loss_function = nn.CrossEntropyLoss(ignore_index=0)
enc_optimizer = optim.Adam(encoder.parameters(),lr=LR)
dec_optimizer = optim.Adam(decoder.parameters(),lr=LR)

In [21]:
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        inputs,targets = pad_to_batch(batch,source2index,target2index)
        
        input_mask = torch.cat([Variable(ByteTensor(tuple(map(lambda s: s ==0, t.data))),volatile=False) for t in inputs]).view(inputs.size(0),-1)
        start_decode = Variable(LongTensor([[target2index['<s>']]*targets.size(0)])).transpose(0,1)
        
        encoder.zero_grad()
        decoder.zero_grad()
        output, hidden_c = encoder(inputs,input_mask)
        
        preds = decoder(start_decode,hidden_c,targets.size(1),output,input_mask,True)
                                
        loss = loss_function(preds,targets.view(-1))
        losses.append(loss.data.cpu().numpy()[0] if USE_CUDA else loss.data.numpy()[0] )
        loss.backward()
        torch.nn.utils.clip_grad_norm(encoder.parameters(), 0.5) # gradient clipping
        torch.nn.utils.clip_grad_norm(decoder.parameters(), 0.5) # gradient clipping
        enc_optimizer.step()
        dec_optimizer.step()

    
        if i % 1000==0:
            print("[%d/%d] [%d/%d] mean_loss : %0.2f" %(step,STEP,i,len(train_data)//BATCH_SIZE,np.mean(losses)))
            torch.save(decoder.state_dict(),os.path.join('../','decoder.pkl'))
            torch.save(encoder.state_dict(),os.path.join('../', 'encoder.pkl'))
            losses=[]

[0/50] [0/4161] mean_loss : 8.95
[0/50] [1000/4161] mean_loss : 6.20
[0/50] [2000/4161] mean_loss : 5.68
[0/50] [3000/4161] mean_loss : 5.31
[0/50] [4000/4161] mean_loss : 5.06
[1/50] [0/4161] mean_loss : 4.88
[1/50] [1000/4161] mean_loss : 4.76
[1/50] [2000/4161] mean_loss : 4.67
[1/50] [3000/4161] mean_loss : 4.58
[1/50] [4000/4161] mean_loss : 4.52
[2/50] [0/4161] mean_loss : 4.33
[2/50] [1000/4161] mean_loss : 4.29
[2/50] [2000/4161] mean_loss : 4.29
[2/50] [3000/4161] mean_loss : 4.27
[2/50] [4000/4161] mean_loss : 4.26
[3/50] [0/4161] mean_loss : 3.75
[3/50] [1000/4161] mean_loss : 4.04
[3/50] [2000/4161] mean_loss : 4.06
[3/50] [3000/4161] mean_loss : 4.08
[3/50] [4000/4161] mean_loss : 4.09
[4/50] [0/4161] mean_loss : 3.98
[4/50] [1000/4161] mean_loss : 3.88
[4/50] [2000/4161] mean_loss : 3.91
[4/50] [3000/4161] mean_loss : 3.94
[4/50] [4000/4161] mean_loss : 3.96
[5/50] [0/4161] mean_loss : 3.66
[5/50] [1000/4161] mean_loss : 3.75
[5/50] [2000/4161] mean_loss : 3.80
[5/50] [30

KeyboardInterrupt: 

### Test

In [68]:
test = random.choice(test_data)
input_ = prepare_sequence(tknz.tokenize(test[0].lower()),word2index).view(1,-1)
input_mask = Variable(ByteTensor(tuple(map(lambda s: s ==0, input_[0].data)))).view(1,-1)
start_decode = Variable(LongTensor([[ta2index['<s>']]*1])).transpose(0,1)
o, hidden_c = encoder(input_,input_mask)
pred = decoder(start_decode,hidden_c,10)
pred = pred.max(1)[1].data.cpu().tolist() if USE_CUDA else pred.max(1)[1].data.tolist()

print(' '.join(test[0]))
print(' '.join(test[1]))
print(' '.join([index2word[t] for t in pred])) 

hi
USER : hi
BOT : i can t know what i don talking about .
do you know kimchi?
USER : do you know kimchi?
BOT : hell ! ! ! the the kimchi !! victoria !
do you know Jisung park?
USER : do you know Jisung park?
BOT : yes . i idea to idea ? about idea ?
hmm
USER : hmm
BOT : it is a matter . i just 't saying .
what is your name
USER : what is your name
BOT : what me t understand :( . i me to discuss
fuck you
USER : fuck you
BOT : yes are are name ? are are are about discuss
hell
USER : hell
BOT : second , go thermodynamics ! the vain , of .
you stupid
USER : you stupid
BOT : i don t know what . i would like .
have a good night
USER : have a good night
BOT : how about we to something to both . about about
fuck..
USER : fuck..
BOT : yeah . i about . your . about . .
are you chatbot?
USER : are you chatbot?
BOT : no ,,!! i 'm... . they by students . i
mm
USER : mm
BOT : you you make the first to your . the iron
bye
USER : bye
BOT : here , ! the ! , . , . ,


because of noisy data, It is hard to converge

# TODO 

* 훈련 제대로 시켜야 한다..(메모리 에러 ㅠ)
* newstest data로 BLEU 측정
* Beam Search

In [22]:
test_en = open('../dataset/iwslt15/tst2012.en','r',encoding='utf-8').readlines()
test_vi = open('../dataset/iwslt15/tst2012.vi','r',encoding='utf-8').readlines()

In [23]:
len(test_en)

1553

In [24]:
len(test_vi)

1553